In [112]:
! pip install -q gpt-2-simple
! pip install datasets
! pip install transformers
! pip install rouge
! pip install bert_score
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
from statistics import mean

In [2]:
gpt2.mount_gdrive()

Mounted at /content/drive


In [3]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [4]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000


In [5]:
context_file_name = "test-input-text.txt"
gpt2.copy_file_from_gdrive(context_file_name)

In [66]:
f = open(context_file_name, 'r')
Lines = f.readlines()

In [67]:
print(Lines[0])

<|startoftext|>admission date : [ 2164/8/31 ] discharge date : [ <H> M <G> 84 <A> black <E> acute pancreatitis | pneumonia, organism unspecified | malignant neoplasm of duodenum | atrial fibrillation | anemia, unspecified | unspecified essential hypertension | pure hypercholesterolemia | coronary atherosclerosis of native coronary artery | old myocardial infarction | hypertrophy (benign) of prostate without urinary obstruction and other lower urinary tract symptom (luts) | personal history of venous thrombosis and embolism | long-term (current) use of anticoagulants <D> <P> lorazepam , 0.5mg Tablet | olanzapine (disintegrating tablet) , 5mg Tablet | lorazepam , 1mg Tablet | morphine sulfate (oral soln.) , 10mg/5mL UDCUP | metoprolol tartrate , 25mg Tablet | lorazepam , 0.5mg Tablet | enoxaparin sodium , 80mg Syringe | enoxaparin sodium , 80mg Syringe <M> mrsa screen : None <T> <L> = 



In [68]:
# !mkdir generate

In [69]:
gen_file_list = []
print("Total Length of Lines: ",len(Lines))
count=1
for f in Lines:
    gen_file = './generate/gpt2_gentext_{:%Y%m%d_%H%M%S}_'.format(datetime.utcnow())+str(count)+'.txt'
    gen_file_list.append(gen_file)
    gpt2.generate_to_file(sess,
                        destination_path=gen_file,
                        length=500,
                        temperature=0.7,
                        truncate='<|endoftext|>',
                        prefix=f[:200],
                        nsamples=3,
                        batch_size=1
                        )
    print("Generated Data from Line "+str(count)+" of test-input.")

Total Length of Lines:  5540
Generated Data from Line 1 of test-input.
Generated Data from Line 2 of test-input.
Generated Data from Line 3 of test-input.


In [61]:
from datasets import load_metric
metric = load_metric('frugalscore.py')

In [70]:
f1 = open(context_file_name,"r")
context = f1.readlines()
best_generation = []

for i in range(len(gen_file_list)):
    f2 = open(gen_file_list[i],"r")
    Lines = f2.readlines()
    predictions = []
    references = []
    for j in range(0,len(Lines),2):
        predictions.append(Lines[j])
        references.append(context[i])
    scores = metric.compute(references=references, predictions=predictions)
    max_index = scores["scores"].index(max(scores["scores"]))
    best_generation.append(Lines[max_index])
    # print("====================================",scores["scores"])

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--moussaKam--frugalscore_tiny_bert-base_bert-score/snapshots/a487e5a875e63ef1f9cf6015a3a11be2d80aa550/config.json
Model config BertConfig {
  "_name_or_path": "moussaKam/frugalscore_tiny_bert-base_bert-score",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights fi

  0%|          | 0/1 [00:00<?, ?ba/s]

Using cuda_amp half precision backend
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2. If sentence1, sentence2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3
  Batch size = 32
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


==================================== [0.9844, 0.9844, 0.9844]


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--moussaKam--frugalscore_tiny_bert-base_bert-score/snapshots/a487e5a875e63ef1f9cf6015a3a11be2d80aa550/config.json
Model config BertConfig {
  "_name_or_path": "moussaKam/frugalscore_tiny_bert-base_bert-score",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights fi

...

==================================== [0.9863, 0.9863, 0.9863]


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--moussaKam--frugalscore_tiny_bert-base_bert-score/snapshots/a487e5a875e63ef1f9cf6015a3a11be2d80aa550/config.json
Model config BertConfig {
  "_name_or_path": "moussaKam/frugalscore_tiny_bert-base_bert-score",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights fi

  0%|          | 0/1 [00:00<?, ?ba/s]

Using cuda_amp half precision backend
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2. If sentence1, sentence2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3
  Batch size = 32
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


==================================== [0.987, 0.987, 0.987]


In [110]:
from datasets import load_metric
metric = load_metric('frugalscore.py')

scores = metric.compute(references=context[:len(best_generation)], predictions=best_generation,
                        pretrained_model_name_or_path="moussaKam/frugalscore_tiny_bert-base_bert-score")
print("Frugal Score (tiny_bert-base_bert-score): ",scores["scores"])

Downloading:   0%|          | 0.00/715 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--moussaKam--frugalscore_tiny_roberta_bert-score/snapshots/eda524feeb885d75d9bb5c948ac224de0ac97bf2/config.json
Model config BertConfig {
  "_name_or_path": "moussaKam/frugalscore_tiny_roberta_bert-score",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Downloading:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--moussaKam--frugalscore_tiny_roberta_bert-score/snapshots/eda524feeb885d75d9bb5c948ac224de0ac97bf2/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at moussaKam/frugalscore_tiny_roberta_bert-score.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


Downloading:   0%|          | 0.00/324 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--moussaKam--frugalscore_tiny_roberta_bert-score/snapshots/eda524feeb885d75d9bb5c948ac224de0ac97bf2/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--moussaKam--frugalscore_tiny_roberta_bert-score/snapshots/eda524feeb885d75d9bb5c948ac224de0ac97bf2/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--moussaKam--frugalscore_tiny_roberta_bert-score/snapshots/eda524feeb885d75d9bb5c948ac224de0ac97bf2/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--moussaKam--frugalscore_tiny_roberta_bert-score/snapshots/eda524feeb885d75d9bb5c948ac224de0ac97bf2/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--moussaKam--frugalscore_tiny_roberta_bert-score/snapshots/eda524feeb885

  0%|          | 0/1 [00:00<?, ?ba/s]

Using cuda_amp half precision backend
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2. If sentence1, sentence2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3
  Batch size = 32
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [111]:
from datasets import load_metric
metric = load_metric('frugalscore.py')

scores = metric.compute(references=context[:len(best_generation)], predictions=best_generation,
                        pretrained_model_name_or_path="moussaKam/frugalscore_tiny_roberta_bert-score")
print("Frugal Score(tiny_roberta_bert-score): ",scores["scores"])

{'scores': [0.9717, 0.971, 0.9766]}


In [ ]:
from datasets import load_metric
metric = load_metric('frugalscore.py')

scores = metric.compute(references=context[:len(best_generation)], predictions=best_generation,
                        pretrained_model_name_or_path="moussaKam/frugalscore_tiny_deberta_bert-score")
print("Frugal Score(tiny_deberta_bert-score): ",scores["scores"])

In [105]:
from datasets import load_metric
bertscore = load_metric("bertscore")

results = bertscore.compute(references=context[:len(best_generation)], predictions=best_generation, lang="en")
print("BERT Score: ")
print(results)

Could not locate the tokenizer configuration file, will try to use the model config instead.


Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/pytorch_model.bin
All the weights of RobertaModel were initialized from the model checkpoint at roberta-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaModel for predictions without further training.


{'precision': [0.8871334791183472, 0.8842194080352783, 0.9042118787765503], 'recall': [0.8212617635726929, 0.7894716858863831, 0.7615019083023071], 'f1': [0.8529276847839355, 0.8341637253761292, 0.8267435431480408], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.24.0)'}


In [99]:
from rouge import Rouge 
scores = []
rouge = Rouge()

for i in range(len(best_generation)):
    hypothesis = best_generation[i]
    reference = context[i]
    scores.append(rouge.get_scores(hypothesis, reference))
print("ROUGE scores:")
print(scores)

In [18]:
import re
line = "[[asassssssd...!@@@~~~     \n\njhjhvjhb"
line = re.sub(r'(\W)(?=\1)', '', line)

In [20]:
print(line)

[asassssssd.!@~ 
jhjhvjhb


In [61]:
text = "sada.@[]vfvsfv#$%**~ ff [12/2/222]   \nr"
text = re.sub('[^a-zA-Z0-9\/\n@.:~\s\[\]]', '',text)
print(text)

sada.@[]vfvsfv~ ff [12/2/222]   
r
